# Montgomery drivers

In [3]:
from datetime import datetime
import re
from tqdm import tqdm

import pandas as pd
import numpy as np
from difflib import get_close_matches

In [4]:
drivers_raw = pd.read_csv("../data/montgomery_drivers.csv")

C:\Users\tymek\AppData\Local\Temp\ipykernel_24240\1728937026.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  drivers_raw = pd.read_csv("../data/montgomery_drivers.csv")


#### utils

In [5]:
def fnv1a_hash_16_digit(s: str) -> int:
    """
    FNV-1a Hash Function to hash a string to a 16-digit deterministic integer value.
    
    :param s: Input string to hash
    :return: Deterministic 16-digit integer hash value
    """
    fnv_prime = 0x1000193
    hash_value = 0xcbf29ce484222325

    for char in s:
        hash_value ^= ord(char)
        hash_value *= fnv_prime
        hash_value &= 0xffffffffffffffff

    return hash_value % 10**16

In [6]:
def describe(column):
    return column.value_counts(dropna=False)

In [7]:
def change_to_unknown(string):
    return 'UNKNOWN' if (string.lower() == 'unknown' or string == 'nan' or string == '') else string

In [8]:
def clean_substance_abuse(substance):
    """ cleans substance abuse column keeping only substance names """
    substance = substance.lower().replace('present', '').replace('contributed', '').replace('detected', '').strip()
    if 'combin' in substance:
        return 'COMBINATION'
    else:
        return substance.upper()

In [9]:
def map_vehicle_type(vehicle):
    """ maps vehicle types to more general ones """
    vehicle = vehicle.lower()
    def in_vehicle(types):
        return any([car_type in vehicle for car_type in types])
    # passenger car
    if in_vehicle(['passenger', 'utility', 'pickup', 'van', 'wagon', 'limousine']) and 'over' not in vehicle:
        return 'PASSENGER'
    elif in_vehicle(['emergency']):
        return 'EMERGENCY'
    elif in_vehicle(['motorcycle', 'moped']):
        return 'MOTORCYCLE'
    elif in_vehicle(['bus']):
        return 'BUS'
    elif in_vehicle(['truck']):
        return 'TRUCK'
    elif in_vehicle(['unknown']):
        return 'UNKNOWN'
    else:
        return 'OTHER'

#### main functions

In [10]:
def prepare_data(data):
    """ selects columns, renames them and changes types """
    data = data[['Report Number', 'Vehicle ID', 'Driver At Fault', 'Injury Severity', 'Driver Substance Abuse', 'Driver Distracted By', 'Vehicle Body Type', 'Vehicle Movement',
                 'Vehicle Going Dir', 'Vehicle Damage Extent', 'Speed Limit', 'Parked Vehicle', 'Vehicle Year', 'Vehicle Make', 'Vehicle Model']]
    data.columns = ['ReportNumber', 'VehicleCrashKey', 'DriverAtFault', 'DriverInjurySeverity', 'DriverSubstanceAbuse', 'DriverDistractedBy', 'VehicleType', 'VehicleMovement',
                 'VehicleGoingDir', 'VehicleDamageExtent', 'SpeedLimit', 'ParkedVehicle', 'VehicleYear', 'VehicleMake', 'VehicleModel']
    data = data.astype(str)
    data['VehicleYear'] = data['VehicleYear'].astype(int)
    return data

In [11]:
def handle_nans(data):
    """ handles na values """
    data = data.copy()
    data['SpeedLimit'] = data['SpeedLimit'].fillna(0)
    data['VehicleYear'] = data['VehicleYear'].fillna(0)
    # str columns changed to unknown
    columns_to_unknown = ['DriverSubstanceAbuse', 'DriverDistractedBy', 'VehicleType', 'VehicleMovement', 'VehicleGoingDir',
                          'VehicleDamageExtent', 'VehicleMake', 'VehicleModel']
    for col in columns_to_unknown:
        data[col] = data[col].apply(change_to_unknown)
    return data

In [40]:
def transform_columns(data):
    """ performs neccessary transformations """
    data = data.copy()
    # clean primary key
    data['VehicleCrashKey'] = data['VehicleCrashKey'].apply(lambda x: x.replace('-', ''))
    # boolean if driver at fault
    data['DriverAtFault'] = data['DriverAtFault'].apply(lambda x: True if x == 'Yes' else False)
    # boolean if substance contributed
    data['SubstanceAbuseContributed'] = data['DriverSubstanceAbuse'].apply(lambda x: True if 'contributed' in x.lower() else False)
    # clean substance
    data['DriverSubstanceAbuse'] = data['DriverSubstanceAbuse'].apply(clean_substance_abuse)    
    # map vehicle types
    data['VehicleType'] = data['VehicleType'].apply(map_vehicle_type)
    # boolean
    data['ParkedVehicle'] = data['ParkedVehicle'].apply(lambda x: True if x == 'Yes' else False)
    # delete impossible year values
    data['VehicleYear'] = data['VehicleYear'].apply(lambda x: 0 if (x < 1900 or x > (datetime.now().year+1)) else x)

    # calculate vehicles crashed total
    crashed_total = data.groupby('ReportNumber').agg(
        VehiclesCrashedTotal=pd.NamedAgg('ReportNumber', 'count')).reset_index()
    data = data.merge(crashed_total, on='ReportNumber')
    return data

In [41]:
drivers_prep = prepare_data(drivers_raw)
drivers_nonan = handle_nans(drivers_prep)
drivers_safe = transform_columns(drivers_nonan)

In [42]:
drivers_safe

,ReportNumber,VehicleCrashKey,DriverAtFault,DriverInjurySeverity,DriverSubstanceAbuse,DriverDistractedBy,VehicleType,VehicleMovement,VehicleGoingDir,VehicleDamageExtent,SpeedLimit,ParkedVehicle,VehicleYear,VehicleMake,VehicleModel,SubstanceAbuseContributed,VehiclesCrashedTotal
0,MCP3170003V,4E492574893B4EB1ADCA53FDD633D6C4,True,NO APPARENT INJURY,NONE,LOOKED BUT DID NOT SEE,PASSENGER,MOVING CONSTANT SPEED,South,FUNCTIONAL,0,False,2017,LEXUS,SUV,False,2
1,MCP3170003V,904F21142B5646D9B105356073FA1D8A,True,NO APPARENT INJURY,NONE,LOOKED BUT DID NOT SEE,BUS,MOVING CONSTANT SPEED,North,NO DAMAGE,0,False,2015,FRHT,BU,False,2
2,MCP3254003K,6D16232C4E1E49A6B3A27FDEF7E506F2,False,NO APPARENT INJURY,NONE,NOT DISTRACTED,PASSENGER,MOVING CONSTANT SPEED,West,FUNCTIONAL,35,False,2010,TOYT,PRIUS,False,1
3,EJ7887003Q,82ED056D33FA44A5AA2B8DCE653FA03C,False,SUSPECTED MINOR INJURY,NONE,NOT DISTRACTED,PASSENGER,MOVING CONSTANT SPEED,South,FUNCTIONAL,35,False,2021,SUBARU,FORRESTER,False,2
4,EJ7887003Q,04B387D7FE5C47B4802A7C8448274F4B,True,NO APPARENT INJURY,NONE,NOT DISTRACTED,PASSENGER,MOVING CONSTANT SPEED,South,FUNCTIONAL,35,False,1991,MERCEDES-BENZ,L350,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172100,MCP1148000V,19FDF33CF67F45198543DF976687A379,True,NO APPARENT INJURY,NONE,NOT DISTRACTED,TRUCK,PARKED,UNKNOWN,SUPERFICIAL,25,True,2014,HONDA,ACCORD,False,1
172101,MCP20080083,6607F24629CA4C5EA956F7B3930387D6,True,NO APPARENT INJURY,NONE,NOT DISTRACTED,PASSENGER,MAKING LEFT TURN,West,SUPERFICIAL,40,False,2018,CHEV,CAMERO,False,1
172102,MCP32710015,D907E819FE51488C931517138E15D026,True,NO APPARENT INJURY,NONE,UNKNOWN,PASSENGER,PARKING,North,SUPERFICIAL,5,False,2015,JEEP,CHEROKEE,False,1
172103,DD5649002Q,76A1A34DB048489DBC22461DD2BB7D35,True,SUSPECTED MINOR INJURY,ALCOHOL,NOT DISTRACTED,PASSENGER,MOVING CONSTANT SPEED,North,DISABLING,25,False,2022,HONDA,ACCORD,True,1


## Vehicles

In [16]:
vehicles_raw = pd.read_csv("../data/vehicles.csv")

C:\Users\tymek\AppData\Local\Temp\ipykernel_24240\1866920876.py:1: DtypeWarning: Columns (72,74,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicles_raw = pd.read_csv("../data/vehicles.csv")


In [17]:
vehicles_raw.columns

Index(['barrels08', 'barrelsA08', 'charge120', 'charge240', 'city08',
       'city08U', 'cityA08', 'cityA08U', 'cityCD', 'cityE', 'cityUF', 'co2',
       'co2A', 'co2TailpipeAGpm', 'co2TailpipeGpm', 'comb08', 'comb08U',
       'combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF', 'cylinders',
       'displ', 'drive', 'engId', 'eng_dscr', 'feScore', 'fuelCost08',
       'fuelCostA08', 'fuelType', 'fuelType1', 'ghgScore', 'ghgScoreA',
       'highway08', 'highway08U', 'highwayA08', 'highwayA08U', 'highwayCD',
       'highwayE', 'highwayUF', 'hlv', 'hpv', 'id', 'lv2', 'lv4', 'make',
       'model', 'mpgData', 'phevBlended', 'pv2', 'pv4', 'range', 'rangeCity',
       'rangeCityA', 'rangeHwy', 'rangeHwyA', 'trany', 'UCity', 'UCityA',
       'UHighway', 'UHighwayA', 'VClass', 'year', 'youSaveSpend', 'baseModel',
       'guzzler', 'trans_dscr', 'tCharger', 'sCharger', 'atvType', 'fuelType2',
       'rangeA', 'evMotor', 'mfrCode', 'c240Dscr', 'charge240b', 'c240bDscr',
       'createdOn'

In [18]:
def prepare_vehicles_data(data):
    data = data[['id', 'make', 'baseModel', 'model', 'year', 'VClass', 'cylinders', 'displ', 'trany', 'drive', 'fuelType1', 'city08', 'highway08']]
    data.columns = ['VehicleKey', 'Make', 'BaseModel', 'Model', 'Year', 'BodyClass', 'Cylinders', 'Displacement', 'Transmission', 'Drivetrain', 'FuelType', 'CityMPG', 'HighwayMPG']
    data.astype(str)
    data.loc[:,['VehicleKey', 'Year', 'Cylinders', 'Displacement', 'CityMPG', 'HighwayMPG']] = data[['VehicleKey', 'Year', 'Cylinders', 'Displacement', 'CityMPG', 'HighwayMPG']].astype(float)
    return data.sort_values('VehicleKey')

In [19]:
def handle_nans_vehicles(data):
    data.loc[:,['Cylinders', 'Displacement']] = data[['Cylinders', 'Displacement']].fillna(0)
    columns_to_fill = ['Make', 'BaseModel', 'Model', 'Year', 'BodyClass', 'Transmission', 'Drivetrain', 'FuelType']
    for col in columns_to_fill:
        data[col] = data[col].fillna('Unknown')
    return data

In [20]:
def transform_transmission(trans):
    if trans == 'Unknown':
        return trans
    try:
        gears = re.findall(r'\d+', trans)[0]
    except Exception:
        gears = 'CVT'
    if 'Automatic' in trans:
        return f"Automatic {gears}"
    elif 'Manual' in trans:
        return f"Manual {gears}"
    else:
        return trans

In [21]:
def transform_drivetrain(drive):
    if drive == "Front-Wheel Drive":
        return 'FWD'
    elif drive == "Rear-Wheel Drive":
        return "RWD"
    elif drive in ["4-Wheel or All-Wheel Drive", "All-Wheel Drive"]:
        return "AWD"
    elif drive in ["4-Wheel Drive", "Part-time 4-Wheel Drive"]:
        return "4WD"
    elif drive == '2-Wheel Drive':
        return "2WD"
    else: 
        return drive

In [22]:
def generate_blank_models(data):
    data = data.copy()
    # brands_dict z car_makes.txt
    makes = list(data['Make'].unique()) + list(brands_dict.values())
    makes_unique = np.unique(makes)
    for make in makes_unique:
        row = {'Make': make, 'Year': 0, 'BaseModel': "Unknown", 'VehicleKey': '', 'BodyClass': '', 'Cylinders': 0, 'Displacement': 0,
               'Transmission': '', 'Drivetrain': '', 'FuelType': '', 'CityMPG': 0,	'HighwayMPG': 0}
        data = pd.concat([data, pd.DataFrame([row])], ignore_index=True)
    return data

In [23]:
def aggregate_models(data):
    return data.drop(['Model'], axis=1).groupby(['Make', 'Year', 'BaseModel']).agg(lambda x: x.mode().iloc[0]).reset_index()

In [30]:
def generate_vehicle_key(make, model, year):
    make = make.replace(' ', '')
    model = model.replace(' ', '')
    return fnv1a_hash_16_digit(f"{make}{model}{year}")

In [31]:
def transform_vehicle_data(data):
    data['Transmission'] = data['Transmission'].apply(transform_transmission)
    data['Drivetrain'] = data['Drivetrain'].apply(transform_drivetrain)
    # generate blank objects for each brand
    data = generate_blank_models(data)
    # aggregate models
    data = aggregate_models(data)
    # generate keys
    data['VehicleKey'] = data.apply(lambda x: generate_vehicle_key(x.Make, x.BaseModel, x.Year), axis=1)
    return data

In [32]:
vehicles_prep = prepare_vehicles_data(vehicles_raw)
vehicles_nan = handle_nans_vehicles(vehicles_prep)
vehicles = transform_vehicle_data(vehicles_nan)

In [33]:
vehicles

,Make,Year,BaseModel,VehicleKey,BodyClass,Cylinders,Displacement,Transmission,Drivetrain,FuelType,CityMPG,HighwayMPG
0,AM General,0,Unknown,7588476098035375,,0.0,0.0,,,,0,0
1,AM General,1984,DJ Po Vehicle,850219001214324,Special Purpose Vehicle 2WD,4.0,2.5,Automatic 3,2WD,Regular Gasoline,18,17
2,AM General,1984,FJ8c Post Office,3248986985309296,Special Purpose Vehicle 2WD,6.0,4.2,Automatic 3,2WD,Regular Gasoline,13,13
3,AM General,1985,Post Office DJ5,8968678517256313,Special Purpose Vehicle 2WD,4.0,2.5,Automatic 3,RWD,Regular Gasoline,16,17
4,AM General,1985,Post Office DJ8,3872758082313622,Special Purpose Vehicle 2WD,6.0,4.2,Automatic 3,RWD,Regular Gasoline,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...
11632,smart,2015,fortwo,7884345398630669,Two Seaters,0.0,0.0,Automatic 1,RWD,Electricity,34,38
11633,smart,2016,fortwo,7884345348297812,Two Seaters,0.0,0.0,Automatic 1,RWD,Electricity,122,39
11634,smart,2017,fortwo,7884345365075431,Two Seaters,3.0,0.9,Automatic 1,RWD,Premium Gasoline,31,38
11635,smart,2018,fortwo,7884345180521622,Two Seaters,0.0,0.0,Automatic 1,RWD,Electricity,112,91


#### Creating mappers

In [149]:
make_counts = drivers['VehicleMake'].value_counts()
filtered_drivers = drivers[drivers['VehicleMake'].isin(make_counts[make_counts > 10].index)]

In [151]:
filtered_drivers['VehicleMake'].value_counts()

VehicleMake
TOYOTA           23171
HONDA            18870
FORD             17138
TOYT              8840
NISSAN            8525
                 ...  
POSTAL              11
PORCHE              11
FREIGHT LINER       11
TRUCK               11
GENS                11
Name: count, Length: 233, dtype: int64

In [153]:
unique_makes_d = filtered_drivers['VehicleMake'].unique()
with open('unique_makes_to_map.txt', 'w') as file:
    for make in unique_makes_d:
        file.write(f"{make}\n")

In [143]:
unique_makes = vehicles['Make'].unique()
with open('unique_makes.txt', 'w') as file:
    for make in unique_makes:
        file.write(f"{make}\n")

### Mapping

In [35]:
""" create dictionary for searching  """
cars_mapper = pd.read_csv("../data/car_makes.txt")
brands_dict = cars_mapper.set_index('unique_makes_to_map')['unique_makes'].to_dict()

In [36]:
""" create dictionary for searching models 

aggregated vehicles data with columns: year, make, basemodel needs to be kept as static file and updated when new cars are fetched.
for example load to dataframe, concatenate with new ones, drop duplicates and save again. also if in new data there are duplicates 
(make/model/year - can be checked with out static file) then drop them. 
"""

models_dict = {}

for index, row in tqdm(vehicles.iterrows()):
    
    key = (row['Year'], row['Make'])
    
    if key not in models_dict:
        models_dict[key] = []
    
    models_dict[key].append(row['BaseModel'])

11637it [00:00, 15389.59it/s]


In [37]:
def map_makes(make):
    """ maps crash data brands to fueleconomy car brands"""
    try:
        new_make = brands_dict[make]
    except KeyError:
        new_make = 'No match found'
        
    if new_make == 'No match found':
        makes_lower = [m.lower() for m in list(brands_dict.values())]
        found_makes = get_close_matches(make.lower(), makes_lower, n=1, cutoff=0.5)
        
        if len(found_makes) == 0:
            new_make = 'Unknown'
        else:
            found_make_lower = found_makes[0]
            found_make_original = list(brands_dict.values())[makes_lower.index(found_make_lower)]
            return found_make_original
            
    return new_make

In [38]:
def map_models(model, make, year):
    """ maps crash data models to fueleconomy car models """
    if model.lower() in ['4s', 'tk']:
        return 'Unknown'

    try:
        models_raw = list(set(models_dict[(year, make)]))
        models_lower = [m.lower() for m in models_raw]
        found_models_lower = get_close_matches(model.lower(), models_lower, n=1, cutoff=0.2)
    except KeyError:
        return 'Unknown'

    # if make == 'BMW' and model only numbers, take first digit. map 3 series -> 3 etc 
    
    if len(found_models_lower) == 0:
        return 'Unknown'
        
    found_model_lower = found_models_lower[0]
    found_model_original = models_raw[models_lower.index(found_model_lower)]
    return found_model_original


In [39]:
def map_year(year, model):
    if model == 'Unknown':
        return 0
    else:
        return year

#### mapping pipeline

In [43]:
def mapping_pipeline(data):
    data = data.copy()
    data['MappedMake'] = data['VehicleMake'].apply(map_makes)
    data['MappedModel'] = data.apply(lambda x: map_models(x.VehicleModel, x.MappedMake, x.VehicleYear), axis=1)
    data['MappedYear'] = data.apply(lambda x: map_year(x.VehicleYear, x.MappedModel), axis=1)
    data['VehicleKey'] = data.apply(lambda x: generate_vehicle_key(x.MappedMake, x.MappedModel, x.MappedYear), axis=1)
    data = data.drop(['VehicleYear', 'VehicleMake', 'VehicleModel', 'MappedMake', 'MappedModel', 'MappedYear'], axis=1)
    return data

In [44]:
drivers_pipe = mapping_pipeline(drivers_safe)

In [46]:
drivers_pipe.head(3)

,ReportNumber,VehicleCrashKey,DriverAtFault,DriverInjurySeverity,DriverSubstanceAbuse,DriverDistractedBy,VehicleType,VehicleMovement,VehicleGoingDir,VehicleDamageExtent,SpeedLimit,ParkedVehicle,SubstanceAbuseContributed,VehiclesCrashedTotal,VehicleKey
0,MCP3170003V,4E492574893B4EB1ADCA53FDD633D6C4,True,NO APPARENT INJURY,NONE,LOOKED BUT DID NOT SEE,PASSENGER,MOVING CONSTANT SPEED,South,FUNCTIONAL,0,False,False,2,6880218977007057
1,MCP3170003V,904F21142B5646D9B105356073FA1D8A,True,NO APPARENT INJURY,NONE,LOOKED BUT DID NOT SEE,BUS,MOVING CONSTANT SPEED,North,NO DAMAGE,0,False,False,2,2250767647570290
2,MCP3254003K,6D16232C4E1E49A6B3A27FDEF7E506F2,False,NO APPARENT INJURY,NONE,NOT DISTRACTED,PASSENGER,MOVING CONSTANT SPEED,West,FUNCTIONAL,35,False,False,1,8224255986863795


In [57]:
vehicles

,Make,Year,BaseModel,VehicleKey,BodyClass,Cylinders,Displacement,Transmission,Drivetrain,FuelType,CityMPG,HighwayMPG
0,AM General,0,Unknown,7588476098035375,,0.0,0.0,,,,0,0
1,AM General,1984,DJ Po Vehicle,850219001214324,Special Purpose Vehicle 2WD,4.0,2.5,Automatic 3,2WD,Regular Gasoline,18,17
2,AM General,1984,FJ8c Post Office,3248986985309296,Special Purpose Vehicle 2WD,6.0,4.2,Automatic 3,2WD,Regular Gasoline,13,13
3,AM General,1985,Post Office DJ5,8968678517256313,Special Purpose Vehicle 2WD,4.0,2.5,Automatic 3,RWD,Regular Gasoline,16,17
4,AM General,1985,Post Office DJ8,3872758082313622,Special Purpose Vehicle 2WD,6.0,4.2,Automatic 3,RWD,Regular Gasoline,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...
11632,smart,2015,fortwo,7884345398630669,Two Seaters,0.0,0.0,Automatic 1,RWD,Electricity,34,38
11633,smart,2016,fortwo,7884345348297812,Two Seaters,0.0,0.0,Automatic 1,RWD,Electricity,122,39
11634,smart,2017,fortwo,7884345365075431,Two Seaters,3.0,0.9,Automatic 1,RWD,Premium Gasoline,31,38
11635,smart,2018,fortwo,7884345180521622,Two Seaters,0.0,0.0,Automatic 1,RWD,Electricity,112,91


In [48]:
vehicles.to_csv("../data/dwh/VehicleDim.csv", index=False)